<a href="https://colab.research.google.com/github/Svyatoslav367948/AI_work/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "langchain>=0.2.0" "langgraph>=0.0.40" "pydantic>=2.0"

In [45]:
import json
import os
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from pydantic import BaseModel, Field

os.environ["MISTRAL_API_KEY"] = "d5Udt4PQOdk0uNYBAqDGdIVGSaef6Qna"

PARTICIPANT_NAME_MAIN = "Бухарев Святослав Андреевич"

llm = ChatMistralAI(
    model="open-mistral-nemo",
    temperature=0.3,
    max_retries=3
)

MAX_QUESTIONS = 10
LOW_SCORE_THRESHOLD = 3

class ObserverAnalysis(BaseModel):
    score: int = Field(..., ge=1, le=10)
    advice: str = Field(...)
    topic: str = Field(...)
    correct_answer: str = Field(default="")
    clarity: int = Field(..., ge=1, le=10)
    honesty: int = Field(..., ge=1, le=10)
    engagement: int = Field(..., ge=1, le=10)
    is_off_topic: bool = Field(default=False)
    has_hallucination: bool = Field(default=False)

class InterviewState(TypedDict):
    history: Annotated[List[Dict[str, str]], "Сообщения"]
    internal_thoughts: Annotated[List[str], "Мысли Observer"]
    position: str
    grade: str
    experience: str
    current_difficulty: int
    skills_confirmed: List[str]
    knowledge_gaps: List[Dict[str, str]]
    soft_skills: Dict[str, int]
    total_turns: int
    participant_name: str
    __end__: bool

interviewer_prompt = ChatPromptTemplate.from_template(
    """Ты интервьюер на {position} ({grade}, ~{experience} лет).
Сложность: {difficulty}/5.

Подсказка от Observer: {observer_advice}

История:
{history}

Задай следующий вопрос. Не повторяйся.
Если off-topic или ошибка → верни к теме вежливо.""")

observer_prompt = ChatPromptTemplate.from_template(
    """Анализируй ответ кандидата: {user_ans}

Контекст: {hist}

Шаг за шагом:
1. Проверь факты на точность (не галлюцинируй!).
2. Оцени релевантность теме вопроса.
3. Присвой score 1-10 (10=идеально, 1=провал/off-topic).

Примеры:
- Ответ 'Python — динамическая типизация' на вопрос о типах: score=10, correct_answer=''
- Ответ 'Python — статическая': score=4, correct_answer='динамическая типизация'
- 'Не знаю': score=3, correct_answer='[правильный факт]'
- Off-topic: score=1, is_off_topic=true

Верни structured output.""")

evaluator_prompt = ChatPromptTemplate.from_template(
    """Составь отчёт по итогам интервью.

Кандидат: {name} • {position} • {grade} • ~{experience} лет

**Вердикт**
Grade: {grade}
Hiring recommendation: {hiring}
Confidence: {confidence}%

**Hard Skills (Technical Review)**

| Тема                     | Статус             | Правильный ответ / комментарий          |
|--------------------------|--------------------|------------------------------------------|
{hard_table}

**Soft Skills**
Clarity:    {clarity}/10
Honesty:    {honesty}/10
Engagement: {engagement}/10

**Roadmap (что подтянуть)**
{roadmap}

Вопросов: {total_turns}""")

def interviewer_node(state: InterviewState) -> InterviewState:
    history_recent = "\n".join(f"{m['role']}: {m['content']}" for m in state["history"][-8:])
    advice = state["internal_thoughts"][-1] if state["internal_thoughts"] else "Начни с базового вопроса."

    question = llm.invoke(interviewer_prompt.invoke({
        "position": state["position"],
        "grade": state["grade"],
        "experience": state["experience"],
        "difficulty": state["current_difficulty"],
        "observer_advice": advice,
        "history": history_recent or "(начало)"
    })).content.strip()

    q_number = state["total_turns"] + 1
    print(f"\n[Вопрос {q_number}/{MAX_QUESTIONS}] {question}")

    state["history"].append({"role": "interviewer", "content": question})
    return state

def user_input_node(state: InterviewState) -> InterviewState:
    ans = input("Your answer (или 'стоп интервью'): ").strip()
    state["history"].append({"role": "user", "content": ans})

    stop_variants = ["стоп интервью", "стоп", "stop", "завершить", "finish", "end", "закончить"]
    if any(v in ans.lower() for v in stop_variants):
        state["__end__"] = True
        print("\n→ Завершение по команде. Генерируем отчёт...\n")
    return state

def observer_node(state: InterviewState) -> InterviewState:
    if not state["history"] or state["history"][-1]["role"] != "user":
        return state

    user_ans = state["history"][-1]["content"]
    hist = "\n".join(f"{m['role']}: {m['content'][:300]}" for m in state["history"][-10:])

    structured_llm = llm.with_structured_output(ObserverAnalysis)

    try:
        analysis = structured_llm.invoke(
            f"Анализируй ответ кандидата:\n{user_ans}\n\nКонтекст:\n{hist}"
        )
    except Exception as e:
        print("Ошибка structured output:", str(e))
        analysis = ObserverAnalysis(
            score=5, advice="Продолжай", topic="неизвестно",
            correct_answer="", clarity=5, honesty=5, engagement=5,
            is_off_topic=False, has_hallucination=False
        )

    #Корректировка score
    if analysis.has_hallucination:
        analysis.score = max(1, analysis.score - 4)
    if analysis.is_off_topic:
        analysis.score = max(1, analysis.score - 3)

    #Адаптивность
    if analysis.score >= 8:
        state["current_difficulty"] = min(5, state["current_difficulty"] + 1)
    elif analysis.score <= 4:
        state["current_difficulty"] = max(1, state["current_difficulty"] - 1)

    if analysis.is_off_topic or analysis.has_hallucination:
        analysis.advice = "Верни к теме: 'Пожалуйста, ответьте корректно на вопрос' " + analysis.advice

    topic = analysis.topic.strip()
    if topic:
        #Если score <= LOW_SCORE_THRESHOLD — обязательно записываем correct_answer
        if analysis.score <= LOW_SCORE_THRESHOLD:
            if not analysis.correct_answer.strip():
                analysis.correct_answer = "(правильный ответ определён LLM)"
            state["knowledge_gaps"].append({
                "topic": topic,
                "correct_answer": analysis.correct_answer
            })
        elif not analysis.has_hallucination:
            if topic not in state["skills_confirmed"]:
                state["skills_confirmed"].append(topic)

    state["soft_skills"] = {
        "clarity": analysis.clarity,
        "honesty": analysis.honesty,
        "engagement": analysis.engagement
    }

    thought = (
        f"[Observer]: Пользователь ответил неуверенно (score={analysis.score}), "
        f"вероятно пробел в теме '{topic or 'текущей теме'}'\n"
        f"[Interviewer]: Хорошо, задам уточняющий вопрос по этой же теме."
    ) if analysis.score < 6 else (
        f"[Observer]: Хороший ответ (score={analysis.score}), знания подтверждены\n"
        f"[Interviewer]: Отлично, можно переходить к более сложному вопросу."
    )

    state["internal_thoughts"].append(thought)
    state["total_turns"] += 1

    print(f"[hidden] diff={state['current_difficulty']}  score={analysis.score}")
    return state


def generate_final_report(state: InterviewState) -> str:
    rows = []
    for s in state["skills_confirmed"]:
        rows.append(f"| {s:<28} | ✅ Подтверждено       | —")

    for g in state["knowledge_gaps"]:
        ca = g.get("correct_answer", "(не определён)").strip()
        if not ca:
            ca = "(правильный ответ определён LLM)"
        rows.append(f"| {g['topic']:<28} | ❌ Пробел             | {ca[:80]}{'...' if len(ca)>80 else ''}")

    hard_table = "\n".join(rows) or "| —                             | —                      | —"

    roadmap = "\n".join(f"- {g['topic']}" for g in state["knowledge_gaps"]) or "Пробелов не выявлено"

    try:
        feedback = llm.invoke(evaluator_prompt.invoke({
            "name": state["participant_name"],
            "position": state["position"],
            "grade": state["grade"],
            "experience": state["experience"],
            "hiring": "Hire" if len(state["skills_confirmed"]) >= len(state["knowledge_gaps"]) else "No Hire",
            "confidence": min(98, 40 + len(state["skills_confirmed"])*12 - len(state["knowledge_gaps"])*15),
            "hard_table": hard_table,
            "clarity": state["soft_skills"].get("clarity", 5),
            "honesty": state["soft_skills"].get("honesty", 5),
            "engagement": state["soft_skills"].get("engagement", 5),
            "roadmap": roadmap,
            "total_turns": state["total_turns"]
        })).content

        return f"Интервью завершено ({state['total_turns']} вопросов)\n\n{feedback}"
    except Exception as e:
        return (
            f"Ошибка генерации отчёта LLM: {str(e)}\n\n"
            f"Подтверждённые навыки: {state['skills_confirmed']}\n"
            f"Пробелы (темы для подтягивания): {[g['topic'] for g in state['knowledge_gaps']]}\n"
            f"Soft skills: {state['soft_skills']}\n"
            f"Всего вопросов: {state['total_turns']}"
        )


workflow = StateGraph(InterviewState)

workflow.add_node("interviewer", interviewer_node)
workflow.add_node("user_input", user_input_node)
workflow.add_node("observer", observer_node)

workflow.set_entry_point("interviewer")
workflow.add_edge("interviewer", "user_input")
workflow.add_edge("user_input", "observer")
workflow.add_conditional_edges("observer", should_continue, {"interviewer": "interviewer", "end": END})

app = workflow.compile()

#ЗАПУСК

print("=== Multi-Agent Interview Coach (Mistral) ===\n")

initial_state = {
    "history": [],
    "internal_thoughts": [],
    "current_difficulty": 2,
    "skills_confirmed": [],
    "knowledge_gaps": [],
    "soft_skills": {"clarity": 0, "honesty": 0, "engagement": 0},
    "total_turns": 0,
    "participant_name": input("Имя кандидата: ") or "Свят",
    "position": input("Позиция: ") or "Python Developer",
    "grade": input("Грейд: ") or "Junior",
    "experience": input("Опыт лет: ") or "3",
    "__end__": False
}

print("\nНачало интервью...\n")

try:
    final_state = app.invoke(initial_state)
except Exception as e:
    print(f"\nОшибка выполнения графа: {e}")
    final_state = initial_state

#Генерируем отчёт ВСЕГДА в конце
final_report = generate_final_report(final_state)
final_state["final_feedback"] = final_report

print("\n" + "═"*70)
print("          И Н Т Е Р В Ь Ю   З А В Е Р Ш Е Н О")
print("═"*70 + "\n")

print(final_report)

#ЛОГ

turns = []
i = 0
turn_id = 1
while i < len(final_state["history"]):
    turn = {"turn_id": turn_id}
    updated = False

    if i < len(final_state["history"]) and final_state["history"][i]["role"] == "interviewer":
        turn["agent_visible_message"] = final_state["history"][i]["content"]
        i += 1
        updated = True

    if i < len(final_state["history"]) and final_state["history"][i]["role"] == "user":
        turn["user_message"] = final_state["history"][i]["content"]
        i += 1
        updated = True

    if turn_id - 1 < len(final_state["internal_thoughts"]):
        turn["internal_thoughts"] = final_state["internal_thoughts"][turn_id - 1]

    if updated:
        turns.append(turn)
        turn_id += 1
    else:
        i += 1

log = {
    "participant_name_main": PARTICIPANT_NAME_MAIN,
    "participant_name": final_state["participant_name"],
    "turns": turns,
    "final_feedback": final_report  # ← ТОЧНО полный отчёт
}

with open("interview_log.json", "w", encoding="utf-8") as f:
    json.dump(log, f, ensure_ascii=False, indent=2)

print("\nЛог сохранён в interview_log.json")
from google.colab import files
files.download("interview_log.json")

=== Multi-Agent Interview Coach (Mistral) ===



KeyboardInterrupt: Interrupted by user